In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import matplotlib.pyplot as plt 
from sklearn.impute import SimpleImputer


import seaborn as sns
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sample_submission.csv
/kaggle/input/sample_submission.csv.gz
/kaggle/input/train.csv.gz
/kaggle/input/data_description.txt
/kaggle/input/test.csv.gz
/kaggle/input/train.csv
/kaggle/input/test.csv


In [2]:
train_df = pd.read_csv('/kaggle/input/train.csv')
test_y_df = pd.read_csv('/kaggle/input/sample_submission.csv')
test_df = pd.read_csv('/kaggle/input/test.csv')

In [3]:
def PrintMissingforEachColumns(data):
    total = data.isnull().sum().sort_values(ascending = False)
    percent_1 = data.isnull().sum()/data.isnull().count()*100
    missing_data = pd.concat([total,percent_1],axis=1,keys=['Total','%'])
    print(missing_data.head())

In [4]:
PrintMissingforEachColumns(train_df)

             Total          %
PoolQC        1453  99.520548
MiscFeature   1406  96.301370
Alley         1369  93.767123
Fence         1179  80.753425
FireplaceQu    690  47.260274


In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Id             1460 non-null   int64  
 1   MSSubClass     1460 non-null   int64  
 2   MSZoning       1460 non-null   object 
 3   LotFrontage    1201 non-null   float64
 4   LotArea        1460 non-null   int64  
 5   Street         1460 non-null   object 
 6   Alley          91 non-null     object 
 7   LotShape       1460 non-null   object 
 8   LandContour    1460 non-null   object 
 9   Utilities      1460 non-null   object 
 10  LotConfig      1460 non-null   object 
 11  LandSlope      1460 non-null   object 
 12  Neighborhood   1460 non-null   object 
 13  Condition1     1460 non-null   object 
 14  Condition2     1460 non-null   object 
 15  BldgType       1460 non-null   object 
 16  HouseStyle     1460 non-null   object 
 17  OverallQual    1460 non-null   int64  
 18  OverallC

## Train Test Split

In [6]:
train_df = train_df.drop(train_df[(train_df['OverallQual']>9) & (train_df['SalePrice']<220000)].index)
train_df = train_df.drop(train_df[(train_df['GrLivArea']>4000) & (train_df['SalePrice']<300000)].index)
Y_train = train_df.SalePrice
train_df = train_df.drop(["SalePrice","Id"],axis=1)
test_df = test_df.drop("Id",axis=1)


In [7]:
from sklearn.model_selection import train_test_split
train_X, val_X, train_y, val_y = train_test_split(train_df, Y_train, random_state=1)

In [8]:
from sklearn.impute import SimpleImputer
def Impute(Train,Test,objectCols,dtype = "obj"):
    Train = Train.copy()
    Test = Test.copy()
    if dtype == "object":
        string = "most_frequent"
    elif dtype == "number":
        string = "mean"
    my_imputer = SimpleImputer(missing_values=np.nan, strategy=string)
    imputed_X_train = pd.DataFrame(my_imputer.fit_transform(Train[objectCols]))
    imputed_X_valid = pd.DataFrame(my_imputer.transform(Test[objectCols]))
    imputed_X_train.columns = Train[objectCols].columns
    imputed_X_valid.columns = Test[objectCols].columns
    Train[objectCols] = imputed_X_train[objectCols].values
    Test[objectCols] = imputed_X_valid[objectCols].values
    return Train, Test

In [9]:
s = (train_X.dtypes == 'object')
n = (train_X.dtypes != 'object')
object_cols = list(s[s].index)
num_cols = list(n[n].index)
print("Categorical variables:")
print(object_cols)

Categorical variables:
['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']


In [10]:
imputed_X_train,imputed_X_valid = Impute(train_X,val_X,object_cols,dtype='object')
imputed_X_train,imputed_X_valid = Impute(imputed_X_train,imputed_X_valid,num_cols,dtype='number')
imputed_X_train.head(),imputed_X_valid.head()

(      MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
 350        120.0       RL    68.000000   7820.0   Pave  Grvl      IR1   
 1350        90.0       RL    91.000000  11643.0   Pave  Grvl      Reg   
 791         80.0       RL    70.037946  11333.0   Pave  Grvl      IR1   
 202         50.0       RL    50.000000   7000.0   Pave  Grvl      Reg   
 1453        20.0       RL    90.000000  17217.0   Pave  Grvl      Reg   
 
      LandContour Utilities LotConfig  ... ScreenPorch PoolArea PoolQC  Fence  \
 350          Lvl    AllPub    Inside  ...         0.0      0.0     Ex  MnPrv   
 1350         Lvl    AllPub    Inside  ...         0.0      0.0     Ex  MnPrv   
 791          Lvl    AllPub    Corner  ...         0.0      0.0     Ex  MnPrv   
 202          Lvl    AllPub    Corner  ...         0.0      0.0     Ex  MnPrv   
 1453         Lvl    AllPub    Inside  ...         0.0      0.0     Ex  MnPrv   
 
      MiscFeature MiscVal  MoSold  YrSold  SaleType  SaleCondition

In [11]:
PrintMissingforEachColumns(imputed_X_train)

              Total    %
MSSubClass        0  0.0
BedroomAbvGr      0  0.0
GarageType        0  0.0
FireplaceQu       0  0.0
Fireplaces        0  0.0


In [12]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

def NominalConvert(Train,Test,object_cols):
    Train = Train.copy()
    Test = Test.copy()

    # Apply ordinal encoder to each column with categorical data
    ordinal_encoder = OrdinalEncoder()
    Train[object_cols] = ordinal_encoder.fit_transform(Train[object_cols])
    Test[object_cols] = ordinal_encoder.fit_transform(Test[object_cols])
    return Train, Test

In [13]:
imputed_X_train, imputed_X_valid = NominalConvert(imputed_X_train,imputed_X_valid,object_cols)
imputed_X_train,imputed_X_valid

(      MSSubClass  MSZoning  LotFrontage  LotArea  Street  Alley  LotShape  \
 350        120.0       3.0    68.000000   7820.0     1.0    0.0       0.0   
 1350        90.0       3.0    91.000000  11643.0     1.0    0.0       3.0   
 791         80.0       3.0    70.037946  11333.0     1.0    0.0       0.0   
 202         50.0       3.0    50.000000   7000.0     1.0    0.0       3.0   
 1453        20.0       3.0    90.000000  17217.0     1.0    0.0       3.0   
 ...          ...       ...          ...      ...     ...    ...       ...   
 716         70.0       4.0    60.000000  10800.0     1.0    0.0       3.0   
 906         20.0       3.0   116.000000  13501.0     1.0    0.0       0.0   
 1097       120.0       3.0    70.037946   3696.0     1.0    0.0       3.0   
 235        160.0       4.0    21.000000   1680.0     1.0    0.0       3.0   
 1062       190.0       4.0    85.000000  13600.0     1.0    0.0       3.0   
 
       LandContour  Utilities  LotConfig  ...  ScreenPorch  Po

In [14]:
from sklearn import preprocessing
def normal(Train,Test):
    Train = Train.copy()
    Test = Test.copy()
    min_max_scaler = preprocessing.MaxAbsScaler()
    x_scaled = min_max_scaler.fit_transform(Train)
    Normal_imputed_X_train = pd.DataFrame(x_scaled)
    Normal_imputed_X_train.columns = Train.columns
    x_scaled = min_max_scaler.transform(Test)
    Normal_imputed_X_valid = pd.DataFrame(x_scaled)
    Normal_imputed_X_valid.columns = Test.columns
    return Normal_imputed_X_train, Normal_imputed_X_valid

In [15]:
Normal_imputed_X_train, Normal_imputed_X_valid = normal(imputed_X_train,imputed_X_valid)
Normal_imputed_X_train.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,0.631579,0.75,0.217252,0.036331,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.666667,1.0,0.0,1.000000,0.998507,0.75,1.0
1,0.473684,0.75,0.290735,0.054092,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.666667,1.0,0.0,0.666667,0.999502,1.00,0.8
2,0.421053,0.75,0.223763,0.052652,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.666667,1.0,0.0,0.416667,0.998507,1.00,0.8
3,0.263158,0.75,0.159744,0.032521,1.0,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.666667,1.0,0.0,0.416667,0.998010,0.00,0.8
4,0.105263,0.75,0.287540,0.079988,1.0,0.0,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.666667,1.0,0.0,0.583333,0.998010,1.00,0.0


## Random Forest

In [16]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(Normal_imputed_X_train, train_y)
rf_val_predictions = rf_model.predict(Normal_imputed_X_valid)
rf_val_mae = mean_absolute_error(rf_val_predictions, val_y)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 16,642


In [17]:
imputed_X_train,imputed_X_valid = Impute(train_df,test_df,object_cols,dtype='object')
imputed_X_train,imputed_X_valid = Impute(imputed_X_train,imputed_X_valid,num_cols,dtype='number')
imputed_X_train, imputed_X_valid = NominalConvert(imputed_X_train,imputed_X_valid,object_cols)
Normal_imputed_X_train, Normal_imputed_X_valid = normal(imputed_X_train,imputed_X_valid)

In [18]:
# Define a random forest model
rf_model = RandomForestRegressor(random_state=1)
rf_model.fit(Normal_imputed_X_train, Y_train)
rf_val_predictions = rf_model.predict(Normal_imputed_X_valid)
rf_val_mae = mean_absolute_error(rf_val_predictions, test_y_df.SalePrice)

print("Validation MAE for Random Forest Model: {:,.0f}".format(rf_val_mae))

Validation MAE for Random Forest Model: 53,816


In [19]:
from xgboost import XGBRegressor
model_1 = XGBRegressor(random_state=0)
model_1.fit(Normal_imputed_X_train, Y_train)
model_val_predictions = model_1.predict(Normal_imputed_X_valid)
model_val_mae = mean_absolute_error(model_val_predictions, test_y_df.SalePrice)
print("Validation MAE for Random Forest Model: {:,.0f}".format(model_val_mae))

Validation MAE for Random Forest Model: 55,205


# Generate a submission

Run the code cell below to generate a CSV file with your predictions that you can use to submit to the competition.

In [20]:
# Run the code to save predictions in the format used for competition scoring

output = pd.DataFrame({'Id': test_y_df.Id,
                       'SalePrice': rf_val_predictions})
output.to_csv('submission.csv', index=False)
print(output)

        Id  SalePrice
0     1461  126290.76
1     1462  156079.00
2     1463  181621.90
3     1464  185203.37
4     1465  194192.56
...    ...        ...
1454  2915   85606.00
1455  2916   85942.11
1456  2917  153364.50
1457  2918  112366.35
1458  2919  234123.05

[1459 rows x 2 columns]
